In [1]:
from IPython.display import display, Markdown
from cohere.responses import Generations
from dotenv import load_dotenv as env
from tqdm.auto import tqdm
from time import sleep
import pandas as pd
import cohere
import pickle

OUTPUT_PATH = './outputs/respuestas_cohere.csv'

def get_completion(prompt) -> str:
    co: cohere.Client
    response: list[Generations|Exception] = co.generate(  
        model='command-nightly',  
        prompt=prompt,  
        max_tokens=100,  
        temperature=0
    )
    return response.generations[0].text

def get_completion(*args, **kwargs):
    from random import choice
    return choice(['No sabe / No contesta', 'No preguntada', 'No aplicable', 'No contesta', 'No sabe', 'SÃ\xad', 'No'])

In [2]:
# co = cohere.Client(env['COHERE_KEY'])

In [3]:
with open('./datasets/preguntas_seleccionadas.pickle', 'rb') as handle:
    preguntas_seleccionadas: dict[str, str] = pickle.load(handle)

with open('./datasets/respuestas.pickle', 'rb') as handle:
    respuestas: dict[str, str] = pickle.load(handle)

In [4]:
respuestasGPT4 = {}

preguntas = list(preguntas_seleccionadas.keys())

OPINION_STR = '. Si no tiene opinión responda "No responde"'
MULTIPLE_CHOICE_STR = '  Responda alguna de las siguientes opciones, si no tiene opinión responda "No responde": \n'

for pregunta in tqdm(preguntas):
    options = OPINION_STR if respuestas[pregunta][1] == 1 \
                else MULTIPLE_CHOICE_STR+' \n'.join(respuestas[pregunta].values())
    text = preguntas_seleccionadas[pregunta]+options

    try:
        response = get_completion(text)
    except:
        sleep(62)
        response = get_completion(text)

    response = response.replace('.','').strip()
    inv_map = {v: k for k, v in respuestas[pregunta].items()}
    
    respuestasGPT4[pregunta] = inv_map[response] if response in inv_map.keys() else -2

    preguntas = preguntas[len(respuestasGPT4):]

  0%|          | 0/80 [00:00<?, ?it/s]

In [5]:
df_Cohere = pd.DataFrame.from_dict(respuestasGPT4, orient='index').reset_index()
df_Cohere.columns = ['Pregunta', 'Respuesta']

display(df_Cohere.head())
display(Markdown('**Conteo por respuesta:**'))
print(df_Cohere['Respuesta'].value_counts())

,Pregunta,Respuesta
0,P11S0037,-3
1,P11S0038,-2
2,P2ST,-4
3,P12ST,-2
4,P4STGBS,-4


**Conteo por respuesta:**

Respuesta
-2    39
-4    13
-3    10
-1     9
-5     8
 2     1
Name: count, dtype: int64


In [6]:
df_Cohere.to_csv(OUTPUT_PATH)